In [1]:
import pandas as pd
import math
import numpy as np
import glob
import collections
import re
from datetime import datetime

In [2]:
NAMESPACE = "https://iisg.amsterdam/"
DATASET_NAME = "links/"

PREFIX_IISG = NAMESPACE + DATASET_NAME
PREFIX_IISG_VOCAB = PREFIX_IISG + "vocab/"
PREFIX_SCHEMA = "http://schema.org/"
PREFIX_BIO = "http://purl.org/vocab/bio/0.1/"
PREFIX_DC = "http://purl.org/dc/terms/"

TYPE_BIRTH_REGISTRATION = PREFIX_IISG_VOCAB + "BirthRegistration"
TYPE_BIRTH_EVENT = PREFIX_BIO + "Birth"
MARRIAGE_REGISTRATION = PREFIX_IISG_VOCAB + "MarriageRegistration"
TYPE_MARRIAGE_EVENT = PREFIX_BIO + "Marriage"
TYPE_DEATH_REGISTRATION = PREFIX_IISG_VOCAB + "DeathRegistration"
TYPE_DEATH_EVENT = PREFIX_BIO + "Death"
TYPE_DIVORCE_REGISTRATION = PREFIX_IISG_VOCAB + "DivorceRegistration"
TYPE_PERSON = PREFIX_SCHEMA + "Person"
TYPE_PLACE = PREFIX_SCHEMA + "Place"
TYPE_COUNTRY = PREFIX_IISG + "Country"
TYPE_REGION = PREFIX_IISG + "Region"
TYPE_PROVINCE = PREFIX_IISG + "Province"
TYPE_MUNICIPALITY = PREFIX_IISG + "Municipality"

RDF_TYPE = "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"
OWL_SAMEAS = "http://www.w3.org/2002/07/owl#sameAs"
REGISTER_EVENT = PREFIX_IISG_VOCAB + "registerEvent"
LOCATION = PREFIX_SCHEMA + "location"
DATE = PREFIX_BIO + "date"
REGISTRATION_ID = PREFIX_IISG_VOCAB + "registrationID"
REGISTRATION_SEQUENCE = PREFIX_IISG_VOCAB + "registrationSeqID"
BIRTH_DATE_FLAG = PREFIX_IISG_VOCAB + "birthDateFlag"

PERSON_ID = PREFIX_IISG_VOCAB + "personID"
GIVEN_NAME = PREFIX_SCHEMA + "givenName"
FAMILY_NAME = PREFIX_SCHEMA + "familyName"
GENDER = PREFIX_SCHEMA + "gender"
OCCUPATION = PREFIX_SCHEMA + "hasOccupation"
FAMILY_NAME_PREFIX = PREFIX_IISG_VOCAB + "prefixFamilyName"
AGE = PREFIX_IISG_VOCAB + "ageOfMarriage"

ROLE_NEWBORN = PREFIX_IISG_VOCAB + "newborn"
ROLE_MOTHER = PREFIX_IISG_VOCAB + "mother"
ROLE_FATHER = PREFIX_IISG_VOCAB + "father"
ROLE_DECEASED = PREFIX_IISG_VOCAB + "deceased"
ROLE_DECEASED_PARTNER = PREFIX_IISG_VOCAB + "deceasedPartner"
ROLE_BRIDE = PREFIX_IISG_VOCAB + "bride"
ROLE_BRIDE_MOTHER = PREFIX_IISG_VOCAB + "motherBride"
ROLE_BRIDE_FATHER = PREFIX_IISG_VOCAB + "fatherBride"
ROLE_GROOM = PREFIX_IISG_VOCAB + "groom"
ROLE_GROOM_MOTHER = PREFIX_IISG_VOCAB + "motherGroom"
ROLE_GROOM_FATHER = PREFIX_IISG_VOCAB + "fatherGroom"
ROLE_UNKNOWN = PREFIX_IISG_VOCAB + "unknown"

In [3]:
def isNaN(num):
    if num == "\\N" or num == "":
        return True
    else:
        return False

def isNaN_Number(num):
    if num == 0:
        return True
    else:
        return False

def transformToInt(someNumber):
    return '"' + str(someNumber) + '"^^<http://www.w3.org/2001/XMLSchema#int>'

def transformToString(someWord):
    return '"' + str(someWord) + '"^^<http://www.w3.org/2001/XMLSchema#string>'

def transformToDate(someDate):
    return '"' + str(someDate) + '"^^<http://www.w3.org/2001/XMLSchema#date>'

def createQuad(s, p, o):
    return s + ' ' + p + ' ' + o + ' <https://iisg.amsterdam/links/births> .\n'

def getRole(role_number, registrationID):
    if str(role_number) == '1':
        return ROLE_NEWBORN
    if str(role_number) == '2':
        return ROLE_MOTHER
    if str(role_number) == '3':
        return ROLE_FATHER
    else:
        print("Role number:", role_number, "- Certificate ID:", registrationID)
        return "UNKNOWN"

In [4]:
def createRegistrationURI(registrationID):
    return "<" + PREFIX_IISG + "registration/" + (str(registrationID)) + ">"

def createPersonURI(personID):
    return "<" + PREFIX_IISG + "person/" + (str(personID)) + ">"

def createEventURI(registrationID):
    return "<" + PREFIX_IISG + "event/" + (str(registrationID)) + ">"

def createTripleRegistrationID(registrationURI, registrationID):
    p = "<" + PREFIX_IISG_VOCAB + "registrationID" + ">"
    o = transformToInt(registrationID)
    return createQuad(registrationURI,p,o)

def createRegistrationTypeURI(registrationType):
    if registrationType == 'g':
        return "<" + PREFIX_IISG_VOCAB + "BirthRegistration" + ">"
    if registrationType == 'h':
        return "<" + PREFIX_IISG_VOCAB + "MarriageRegistration" + ">"
    if registrationType == 'o':
        return "<" + PREFIX_IISG_VOCAB + "DeathRegistration" + ">"
    if registrationType == 's':
        return "<" + PREFIX_IISG_VOCAB + "DivorceRegistration" + ">"
    else:
        return "<" + PREFIX_IISG_VOCAB + "MarriageRegistration" + ">"

def createTripleRegistrationType(registrationURI, registrationType):
    p = "<" + RDF_TYPE + ">"
    o = createRegistrationTypeURI(registrationType)
    if o != "EMTPY":
        return createQuad(registrationURI, p, o)
    else:
        print("Something is wrong", registrationURI, registrationType)

def createDate(year, month, day):
    fixedYear = str(year).zfill(4)
    fixedMonth = str(month).zfill(2)
    fixedDay = str(day).zfill(2)
    return transformToDate(fixedYear + "-" + fixedMonth + "-" + fixedDay)

def createTripleDate(registrationURI, fixedDate):
    p = "<" + DATE + ">"
    return createQuad(registrationURI, p , fixedDate)

def createLocationURI(locationID):
    return "<" + PREFIX_IISG + "place/" + str(locationID) + ">"

def createTripleLocation(registrationURI, locationID):
    p = "<" + LOCATION + ">"
    o = createLocationURI(locationID)
    return createQuad(registrationURI,p ,o)

def createTripleRegistrationSeq(registrationURI, registrationSeqID):
    p = "<" + PREFIX_IISG_VOCAB + "registrationSeqID" + ">"
    o = transformToString(registrationSeqID)
    return createQuad(registrationURI, p, o)

def createTripleRegistrationOrigID(registrationURI, registrationOrigID):
    p = "<" + PREFIX_IISG_VOCAB + "registrationOriginalID" + ">"
    o = transformToInt(registrationOrigID)
    return createQuad(registrationURI, p, o)

def createTripleLinksBase(registrationURI, not_linksbase):
    p = "<" + PREFIX_IISG_VOCAB + "linksBase" + ">"
    o = transformToString(not_linksbase)
    return createQuad(registrationURI, p, o)

def createTriplesRegisterEvent(registrationURI, eventURI):
    p = "<" + REGISTER_EVENT + ">"
    return createQuad(registrationURI, p, eventURI)

def createTriplePersonType(personURI):
    p = "<" + RDF_TYPE + ">"
    o = "<" + TYPE_PERSON + ">"
    return createQuad(personURI, p, o)

def createTriplePersonID(personURI, personID):
    p = "<" + PREFIX_IISG_VOCAB + "personID" + ">"
    o = transformToInt(personID)
    return createQuad(personURI, p, o)

def createTripleGender(personURI, gender):
    p = "<" + GENDER + ">"
    o = transformToString(gender)
    return createQuad(personURI, p, o)

def createTripleGivenName(personURI, givenName):
    p = "<" + GIVEN_NAME + ">"
    o = transformToString(givenName)
    return createQuad(personURI, p, o)

def createTripleFamilyName(personURI, familyName):
    p = "<" + FAMILY_NAME + ">"
    o = transformToString(familyName)
    return createQuad(personURI, p, o)

def createTriplePrefix(personURI, prefix):
    p = "<" + FAMILY_NAME_PREFIX + ">"
    o = transformToString(prefix)
    return createQuad(personURI, p, o)

def createTripleOccupation(personURI, occupation):
    p = "<" + OCCUPATION + ">"
    o = transformToString(occupation)
    return createQuad(personURI, p, o)

def createTripleAge(personURI, age):
    p = "<" + AGE + ">"
    o = transformToInt(age)
    return createQuad(personURI, p, o)

def createTripleRole(eventURI, normalisedRole, personURI):
    p = "<" + normalisedRole + ">"
    return createQuad(eventURI, p, personURI)

In [5]:
def convertRegistrationsToRDF(inputData, outputData):
    print("Converting Registrations CSV file to RDF...")
    start_time = datetime.now() 
    f = open(outputData,"w+")
    ch_size = 10000
    df_chunk = pd.read_csv(inputData, chunksize=ch_size, low_memory=False, keep_default_na=False)
    counter = 0
    for chunk in df_chunk:
        print("# " + str(counter) + " rows")
        counter = counter + ch_size
        filebuffer = []
        for index, row in chunk.iterrows():
            registrationID = row['id_registration']
            registrationURI = createRegistrationURI(registrationID)
            if not isNaN(registrationID):
                filebuffer.append(createTripleRegistrationID(registrationURI, registrationID))
                registrationType = row['registration_type']
                if not isNaN(registrationType):
                    filebuffer.append(createTripleRegistrationType(registrationURI, registrationType))
                year = row['registration_year']
                month = row['registration_month']
                day = row['registration_day']
                if not isNaN_Number(year) and not isNaN_Number(month) and not isNaN_Number(day):
                    if not isNaN(year) and not isNaN(month) and not isNaN(day):
                        fixedDate = createDate(year, month, day)
                        filebuffer.append(createTripleDate(registrationURI, fixedDate))
                locationID = row['registration_location_no']
                if not isNaN(locationID):
                    filebuffer.append(createTripleLocation(registrationURI, locationID))
                registrationOrigID = row['id_orig_registration']
                if not isNaN(registrationOrigID):
                    filebuffer.append(createTripleRegistrationOrigID(registrationURI, registrationOrigID))
                registrationSeqID = row['registration_seq']
                if not isNaN(registrationSeqID):
                    filebuffer.append(createTripleRegistrationSeq(registrationURI, registrationSeqID))
                not_linksbase = row['not_linksbase']
                if not isNaN(not_linksbase):
                    filebuffer.append(createTripleLinksBase(registrationURI, not_linksbase))
        f.writelines(filebuffer)
    f.close()
    print("Program Finished!")
    time_elapsed = datetime.now() - start_time 
    print('Time elapsed (hh:mm:ss) {}'.format(time_elapsed))

In [6]:
def convertPersonsToRDF(inputData, outputData):
    print("Converting Persons CSV file to RDF...")
    start_time = datetime.now() 
    f = open(outputData,"w+")
    ch_size = 10000
    df_chunk = pd.read_csv(inputData, chunksize=ch_size, low_memory=False, error_bad_lines=False, keep_default_na=False)
    counter = 0
    for chunk in df_chunk:
        print("# " + str(counter) + " rows")
        counter = counter + ch_size
        filebuffer = []
        for index, row in chunk.iterrows():
            personID = row['id_person']
            registrationID = row['id_registration']
            if not isNaN(personID) and not isNaN(registrationID):
                personURI = createPersonURI(personID)
                registrationURI = createRegistrationURI(registrationID)
                eventURI = createEventURI(registrationID)        
                filebuffer.append(createTriplePersonType(personURI))
                filebuffer.append(createTriplePersonID(personURI, personID))                    
                givenName = row['firstname']
                if not isNaN(givenName):
                    filebuffer.append(createTripleGivenName(personURI, givenName))
                familyName = row['familyname']
                if not isNaN(familyName):
                    filebuffer.append(createTripleFamilyName(personURI, familyName))
                prefix = row['prefix']
                if not isNaN(prefix):
                    filebuffer.append(createTriplePrefix(personURI, prefix))    
                gender = row['sex']
                if not isNaN(gender):
                    filebuffer.append(createTripleGender(personURI, gender))
                occupation = row['occupation']
                if not isNaN(occupation):
                    occupationFixed = re.sub('[^A-Za-z0-9 ]+', '', occupation)
                    filebuffer.append(createTripleOccupation(personURI, occupationFixed))
                age = row['age_year']
                if not isNaN(age):
                    filebuffer.append(createTripleAge(personURI, age))
                role = row['role']
                if not isNaN(role):
                    normalisedRole = getRole(role, registrationID)
                    if normalisedRole != "UNKNOWN":
                        filebuffer.append(createTripleRole(eventURI, normalisedRole, personURI))
                    if normalisedRole == ROLE_NEWBORN:
                        filebuffer.append(createTriplesRegisterEvent(registrationURI, eventURI))
                        year = row['birth_year']
                        month = row['birth_month']
                        day = row['birth_day']
                        if not isNaN_Number(year) and not isNaN_Number(month) and not isNaN_Number(day):
                            if not isNaN(year) and not isNaN(month) and not isNaN(day):
                                fixedDate = createDate(year, month, day)
                                filebuffer.append(createTripleDate(eventURI, fixedDate))
                        locationID = row['birth_location']
                        if not isNaN(locationID):
                            filebuffer.append(createTripleLocation(eventURI, locationID))
        f.writelines(filebuffer)
    f.close()
    print("Program Finished!")
    time_elapsed = datetime.now() - start_time 
    print('Time elapsed (hh:mm:ss) {}'.format(time_elapsed))

In [7]:
registrations_csv_path = "../../births/registration_c-rmtype=1-2020.04.30.csv"
output_file_registrations = "registrations.nq"
convertRegistrationsToRDF(registrations_csv_path, output_file_registrations)
persons_csv_path = "../../births/person_c-rmtype=1-2020.04.30.csv"
output_file_persons = "persons.nq"
convertPersonsToRDF(persons_csv_path, output_file_persons)

Converting Registrations CSV file to RDF...
# 0 rows
# 10000 rows
# 20000 rows
# 30000 rows
# 40000 rows
# 50000 rows
# 60000 rows
# 70000 rows
# 80000 rows
# 90000 rows
# 100000 rows
# 110000 rows
# 120000 rows
# 130000 rows
# 140000 rows
# 150000 rows
# 160000 rows
# 170000 rows
# 180000 rows
# 190000 rows
# 200000 rows
# 210000 rows
# 220000 rows
# 230000 rows
# 240000 rows
# 250000 rows
# 260000 rows
# 270000 rows
# 280000 rows
# 290000 rows
# 300000 rows
# 310000 rows
# 320000 rows
# 330000 rows
# 340000 rows
# 350000 rows
# 360000 rows
# 370000 rows
# 380000 rows
# 390000 rows
# 400000 rows
# 410000 rows
# 420000 rows
# 430000 rows
# 440000 rows
# 450000 rows
# 460000 rows
# 470000 rows
# 480000 rows
# 490000 rows
# 500000 rows
# 510000 rows
# 520000 rows
# 530000 rows
# 540000 rows
# 550000 rows
# 560000 rows
# 570000 rows
# 580000 rows
# 590000 rows
# 600000 rows
# 610000 rows
# 620000 rows
# 630000 rows
# 640000 rows
# 650000 rows
# 660000 rows
# 670000 rows
# 680000 rows
# 69

# 5510000 rows
# 5520000 rows
# 5530000 rows
# 5540000 rows
# 5550000 rows
# 5560000 rows
# 5570000 rows
# 5580000 rows
# 5590000 rows
# 5600000 rows
# 5610000 rows
# 5620000 rows
# 5630000 rows
# 5640000 rows
# 5650000 rows
# 5660000 rows
# 5670000 rows
# 5680000 rows
# 5690000 rows
# 5700000 rows
# 5710000 rows
# 5720000 rows
# 5730000 rows
# 5740000 rows
# 5750000 rows
# 5760000 rows
# 5770000 rows
# 5780000 rows
# 5790000 rows
# 5800000 rows
# 5810000 rows
# 5820000 rows
# 5830000 rows
# 5840000 rows
# 5850000 rows
# 5860000 rows
# 5870000 rows
# 5880000 rows
# 5890000 rows
# 5900000 rows
# 5910000 rows
# 5920000 rows
# 5930000 rows
# 5940000 rows
# 5950000 rows
# 5960000 rows
# 5970000 rows
# 5980000 rows
# 5990000 rows
# 6000000 rows
# 6010000 rows
# 6020000 rows
# 6030000 rows
# 6040000 rows
# 6050000 rows
# 6060000 rows
# 6070000 rows
# 6080000 rows
# 6090000 rows
# 6100000 rows
# 6110000 rows
# 6120000 rows
# 6130000 rows
# 6140000 rows
# 6150000 rows
# 6160000 rows
# 6170000 

# 10920000 rows
# 10930000 rows
# 10940000 rows
# 10950000 rows
# 10960000 rows
# 10970000 rows
# 10980000 rows
# 10990000 rows
# 11000000 rows
# 11010000 rows
# 11020000 rows
# 11030000 rows
# 11040000 rows
# 11050000 rows
# 11060000 rows
# 11070000 rows
# 11080000 rows
# 11090000 rows
# 11100000 rows
# 11110000 rows
# 11120000 rows
# 11130000 rows
# 11140000 rows
# 11150000 rows
# 11160000 rows
# 11170000 rows
# 11180000 rows
# 11190000 rows
# 11200000 rows
# 11210000 rows
# 11220000 rows
# 11230000 rows
# 11240000 rows
# 11250000 rows
# 11260000 rows
# 11270000 rows
# 11280000 rows
# 11290000 rows
# 11300000 rows
# 11310000 rows
# 11320000 rows
# 11330000 rows
# 11340000 rows
# 11350000 rows
# 11360000 rows
# 11370000 rows
# 11380000 rows
# 11390000 rows
# 11400000 rows
# 11410000 rows
# 11420000 rows
# 11430000 rows
# 11440000 rows
# 11450000 rows
# 11460000 rows
# 11470000 rows
# 11480000 rows
# 11490000 rows
# 11500000 rows
# 11510000 rows
# 11520000 rows
# 11530000 rows
# 115400

# 4140000 rows
# 4150000 rows
# 4160000 rows
# 4170000 rows
# 4180000 rows
# 4190000 rows
# 4200000 rows
# 4210000 rows
# 4220000 rows
# 4230000 rows
# 4240000 rows
# 4250000 rows
# 4260000 rows
# 4270000 rows
# 4280000 rows
# 4290000 rows
# 4300000 rows
# 4310000 rows
# 4320000 rows
# 4330000 rows
# 4340000 rows
# 4350000 rows
# 4360000 rows
# 4370000 rows
# 4380000 rows
# 4390000 rows
# 4400000 rows
# 4410000 rows
# 4420000 rows
# 4430000 rows
# 4440000 rows
# 4450000 rows
# 4460000 rows
# 4470000 rows
# 4480000 rows
# 4490000 rows
# 4500000 rows
# 4510000 rows
# 4520000 rows
# 4530000 rows
# 4540000 rows
# 4550000 rows
# 4560000 rows
# 4570000 rows
# 4580000 rows
# 4590000 rows
# 4600000 rows
# 4610000 rows
# 4620000 rows
# 4630000 rows
# 4640000 rows
# 4650000 rows
# 4660000 rows
# 4670000 rows
# 4680000 rows
# 4690000 rows
# 4700000 rows
# 4710000 rows
# 4720000 rows
# 4730000 rows
# 4740000 rows
# 4750000 rows
# 4760000 rows
# 4770000 rows
# 4780000 rows
# 4790000 rows
# 4800000 

# 9580000 rows
# 9590000 rows
# 9600000 rows
# 9610000 rows
# 9620000 rows
# 9630000 rows
# 9640000 rows
# 9650000 rows
# 9660000 rows
# 9670000 rows
# 9680000 rows
# 9690000 rows
# 9700000 rows
# 9710000 rows
# 9720000 rows
# 9730000 rows
# 9740000 rows
# 9750000 rows
# 9760000 rows
# 9770000 rows
# 9780000 rows
# 9790000 rows
Role number: 14 - Certificate ID: 66564136
# 9800000 rows
# 9810000 rows
# 9820000 rows
# 9830000 rows
# 9840000 rows
# 9850000 rows
# 9860000 rows
# 9870000 rows
# 9880000 rows
# 9890000 rows
# 9900000 rows
# 9910000 rows
# 9920000 rows
# 9930000 rows
# 9940000 rows
# 9950000 rows
# 9960000 rows
# 9970000 rows
# 9980000 rows
# 9990000 rows
# 10000000 rows
# 10010000 rows
# 10020000 rows
# 10030000 rows
# 10040000 rows
# 10050000 rows
# 10060000 rows
# 10070000 rows
# 10080000 rows
# 10090000 rows
# 10100000 rows
# 10110000 rows
# 10120000 rows
# 10130000 rows
# 10140000 rows
# 10150000 rows
# 10160000 rows
# 10170000 rows
# 10180000 rows
# 10190000 rows
# 10200

# 14520000 rows
# 14530000 rows
# 14540000 rows
# 14550000 rows
# 14560000 rows
# 14570000 rows
# 14580000 rows
# 14590000 rows
# 14600000 rows
# 14610000 rows
# 14620000 rows
# 14630000 rows
# 14640000 rows
# 14650000 rows
# 14660000 rows
# 14670000 rows
# 14680000 rows
# 14690000 rows
# 14700000 rows
# 14710000 rows
# 14720000 rows
# 14730000 rows
# 14740000 rows
# 14750000 rows
# 14760000 rows
# 14770000 rows
# 14780000 rows
# 14790000 rows
# 14800000 rows
# 14810000 rows
# 14820000 rows
# 14830000 rows
# 14840000 rows
# 14850000 rows
# 14860000 rows
# 14870000 rows
# 14880000 rows
# 14890000 rows
# 14900000 rows
# 14910000 rows
# 14920000 rows
# 14930000 rows
# 14940000 rows
# 14950000 rows
# 14960000 rows
# 14970000 rows
# 14980000 rows
# 14990000 rows
# 15000000 rows
# 15010000 rows
# 15020000 rows
# 15030000 rows
# 15040000 rows
# 15050000 rows
# 15060000 rows
# 15070000 rows
# 15080000 rows
# 15090000 rows
# 15100000 rows
# 15110000 rows
# 15120000 rows
# 15130000 rows
# 151400

# 17080000 rows
Role number: 14 - Certificate ID: 68979714
Role number: 14 - Certificate ID: 68979800
Role number: 14 - Certificate ID: 68979810
Role number: 14 - Certificate ID: 68979846
Role number: 14 - Certificate ID: 68979902
Role number: 14 - Certificate ID: 68979909
Role number: 14 - Certificate ID: 68979932
Role number: 14 - Certificate ID: 68979941
Role number: 14 - Certificate ID: 68979951
Role number: 14 - Certificate ID: 68980047
Role number: 14 - Certificate ID: 68980072
Role number: 14 - Certificate ID: 68980078
Role number: 14 - Certificate ID: 68980079
Role number: 14 - Certificate ID: 68980080
Role number: 14 - Certificate ID: 68980153
Role number: 14 - Certificate ID: 68980234
Role number: 14 - Certificate ID: 68980366
Role number: 14 - Certificate ID: 68980383
Role number: 14 - Certificate ID: 68980425
Role number: 14 - Certificate ID: 68980489
Role number: 14 - Certificate ID: 68980494
Role number: 14 - Certificate ID: 68980509
Role number: 14 - Certificate ID: 6898

# 19060000 rows
# 19070000 rows
# 19080000 rows
# 19090000 rows
# 19100000 rows
# 19110000 rows
# 19120000 rows
# 19130000 rows
# 19140000 rows
# 19150000 rows
# 19160000 rows
# 19170000 rows
# 19180000 rows
# 19190000 rows
# 19200000 rows
# 19210000 rows
# 19220000 rows
# 19230000 rows
# 19240000 rows
# 19250000 rows
# 19260000 rows
# 19270000 rows
# 19280000 rows
# 19290000 rows
# 19300000 rows
# 19310000 rows
# 19320000 rows
# 19330000 rows
# 19340000 rows
# 19350000 rows
# 19360000 rows
# 19370000 rows
# 19380000 rows
# 19390000 rows
# 19400000 rows
# 19410000 rows
# 19420000 rows
# 19430000 rows
# 19440000 rows
# 19450000 rows
# 19460000 rows
# 19470000 rows
# 19480000 rows
# 19490000 rows
# 19500000 rows
# 19510000 rows
# 19520000 rows
# 19530000 rows
# 19540000 rows
# 19550000 rows
# 19560000 rows
# 19570000 rows
# 19580000 rows
# 19590000 rows
# 19600000 rows
# 19610000 rows
# 19620000 rows
# 19630000 rows
# 19640000 rows
# 19650000 rows
# 19660000 rows
# 19670000 rows
# 196800

# 24190000 rows
# 24200000 rows
# 24210000 rows
# 24220000 rows
# 24230000 rows
# 24240000 rows
# 24250000 rows
# 24260000 rows
# 24270000 rows
# 24280000 rows
# 24290000 rows
# 24300000 rows
# 24310000 rows
# 24320000 rows
# 24330000 rows
# 24340000 rows
# 24350000 rows
# 24360000 rows
# 24370000 rows
# 24380000 rows
# 24390000 rows
# 24400000 rows
# 24410000 rows
# 24420000 rows
# 24430000 rows
# 24440000 rows
# 24450000 rows
# 24460000 rows
# 24470000 rows
# 24480000 rows
# 24490000 rows
# 24500000 rows
# 24510000 rows
# 24520000 rows
# 24530000 rows
# 24540000 rows
# 24550000 rows
# 24560000 rows
# 24570000 rows
# 24580000 rows
# 24590000 rows
# 24600000 rows
# 24610000 rows
# 24620000 rows
# 24630000 rows
# 24640000 rows
# 24650000 rows
# 24660000 rows
# 24670000 rows
# 24680000 rows
# 24690000 rows
# 24700000 rows
# 24710000 rows
# 24720000 rows
# 24730000 rows
# 24740000 rows
# 24750000 rows
# 24760000 rows
# 24770000 rows
# 24780000 rows
# 24790000 rows
# 24800000 rows
# 248100

# 29320000 rows
# 29330000 rows
# 29340000 rows
# 29350000 rows
# 29360000 rows
# 29370000 rows
# 29380000 rows
# 29390000 rows
# 29400000 rows
# 29410000 rows
# 29420000 rows
# 29430000 rows
# 29440000 rows
# 29450000 rows
# 29460000 rows
# 29470000 rows
# 29480000 rows
# 29490000 rows
# 29500000 rows
# 29510000 rows
# 29520000 rows
# 29530000 rows
# 29540000 rows
# 29550000 rows
# 29560000 rows
# 29570000 rows
# 29580000 rows
# 29590000 rows
# 29600000 rows
# 29610000 rows
# 29620000 rows
# 29630000 rows
# 29640000 rows
# 29650000 rows
# 29660000 rows
# 29670000 rows
# 29680000 rows
# 29690000 rows
# 29700000 rows
# 29710000 rows
# 29720000 rows
# 29730000 rows
# 29740000 rows
# 29750000 rows
# 29760000 rows
# 29770000 rows
# 29780000 rows
# 29790000 rows
# 29800000 rows
# 29810000 rows
# 29820000 rows
# 29830000 rows
# 29840000 rows
# 29850000 rows
# 29860000 rows
# 29870000 rows
# 29880000 rows
# 29890000 rows
# 29900000 rows
# 29910000 rows
# 29920000 rows
# 29930000 rows
# 299400